# Estimation

In this notebook we provide an estimation method for the confidence intervals of the lower/upper bounds in CHH(2020). We follow the estimation framework in CCT(2018).

## 1. Moments


## 2. Optimally-weighted or continuously-updated sieve GMM estimation with fixed $\theta$


## 3. MC Simulation

Here we seek a $100\alpha \%$ CS $\hat{\Theta}_\alpha$ for $\Theta_I$ using $L_n(\theta)$ that has asymptotically exact coverage, i.e.:

$$
\lim_{n\to \infty} \mathbb{P}(\Theta_I \subseteq \hat{\Theta}_\alpha) = \alpha
$$

1. Draw a sample $\{\theta^1,...,\theta^B\}$ from the quasi-posterior distribution $\Pi_n := \frac{\exp(n L_n(\theta)) \Pi(\theta)}{\int_{\Theta}\exp(n L_n(\theta)\Pi(\theta))}$

2. Calculate the $(1-\alpha)$ quantile of $\{L_n(\theta^1),...,L_n(\theta^B)\}$; call it $\zeta_{n,\alpha}^{mc}$.

3. Our $100\alpha\%$ confidence set for $\Theta_I$ is then:

$$
\hat{\Theta}_\alpha = \{\theta \in \Theta: L_n(\theta)\geq \zeta_{n,\alpha}^{mc}\}
$$

### 3.1. Adaptive Sequential Monte Carlo Algorithm

Let $J$ and $K$ be positive integers and let $\phi_1,...,\phi_J$ be an increasing sequence with $\phi_1=0$ and $\phi_J=1$. Set $w_1^b=1$ for $b=1,...,B$ and draw $\theta_1^1, ..., \theta_1^B$ from the prior $\Pi(\theta)$. 

For $j=2,...,J$, do:

1. Correction: Let $v_j^b = \exp((\phi_j-\phi_{j-1})nL_n(\theta^b_{j-1}))$ and $w_j^b = (v_j^bw_{j-1}^b)/(\frac{1}{B}\sum_{b=1}^Bv_j^bw_{j-1}^b)$.

2. Selection: Compute the effective sample size $ESS_j = B/(\frac{1}{B}\sum_{b=1}^B(w_j^b)^2)$. Then:
    - If $ESS_j>\frac{B}{2}$: set $\ell_j^b = \theta_{j-1}^b$ for $b=1,...,B$; or
    - If $ESS_j\leq \frac{B}{2}$: draw an i.i.d. sample $\ell_j^1,...,\ell_j^B$ from the multinomial distribution with support $\theta_{j-1}^1,...,\theta_{j-1}^B$ and weights $w_j^1,...,w_j^B$, then set $w_j^b=1$ for $b=1,...,B$.


3. Mutation: Run $B$ separate and independent MCMC chains of length $K$ using the random-walk Metropolis-Hastings algorithm initialized at each $\ell_j^b$ for the tempered quasi-posterior $\Pi_j(\theta|\mathbf{X}_n)\propto \exp(\phi_j n L_n(\theta))\Pi(\theta)$ and let $\theta_j^b$ be the final draw of the $b$th chain.

The resulting sample is $\theta^b = \theta^b_J for b=1,...,B$. Multinomial resampling (step 2) and the $B$ independent MCMC chains (step 3) can both be computed in parallel, so the additional computational time relative to conventional MCMC methods is modest.

#### 3.1.1 Metropolis-Hastings

Suppose $q$ is the proposal distribution, $\pi$ is the desired joint distribution. 

Initialize $x_0$. for iteration $i = 1,2,...$ do:
1. Propose: $x^{cand} \sim q\left(x^{(i)}|x^{(i-1)}\right)$
2. Acceptance Probability:
    $$
    \alpha(x^{(cand)}|x^{(i-1)}) = \min \{1, \frac{q\left(x^{(i-1)}|x^{cand}\right)\pi(x^{cand})}{q\left(x^{cand}|x^{(i-1)}\right)\pi(x^{(i-1)})}\}
    $$
3. Draw $u$ from $\text{Uniform }(0,1)$;
4. If $u<\alpha$, then accept the proposal: $x^{(i)} \leftarrow x^{cand}$; Else, reject the proposal: $x^{(i)} \leftarrow x^{(i-1)}$

Note 1: In the applicaton, we use a random walk chain, i.e. $x^{(i)}=x^{(i-1)}+\epsilon$, where $\epsilon$ is a multivariate normal distribution with mean $0$ and covariance matrix $\Sigma$.

Note 2: We can initialize $x_0$ from an arbitrary distribution, for example, the prior.